## **Connect to drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Change the following paths**

In [ ]:
import os

# path where train csv file stored
csv_file_path = '/content/drive/My Drive/Assignments/Deeper_Systems/data/train.truth.csv'


# path of the training data
train_data_path = '/content/drive/My Drive/Assignments/Deeper_Systems/data/train'


# path to save the trained model 
save_model = '/content/drive/My Drive/Assignments/Deeper_Systems/cifar10_cnn'

if not os.path.exists(save_model):
    os.makedirs(save_model)


# path of the testing data
test_path = '/content/drive/My Drive/Assignments/Deeper_Systems/data/test'
test_image_list = os.listdir(test_path)


# path to save the testing images after correcting the orientation
corrected_image_path = '/content/drive/My Drive/Assignments/Deeper_Systems/cifar10_cnn/corrected_images'

if not os.path.exists(corrected_image_path):
    os.makedirs(corrected_image_path)


# path to save the results
path_to_save = '/content/drive/My Drive/Assignments/Deeper_Systems/cifar10_cnn/result'

if not os.path.exists(path_to_save):
    os.makedirs(path_to_save)

## **Libraries**

In [ ]:
import pandas as pd
import os
from skimage import io
import numpy as np
import cv2

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## **Check device**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## **Map labels**

In [ ]:
def map_label(l):
  d = {'rotated_right':0, 'rotated_left':1, 'upright':2, 'upside_down':3}
  return d[l]

In [ ]:
def return_label(l):
  d = {0:'rotated_right', 1:'rotated_left', 2:'upright', 3:'upside_down'}
  return d[l]

## **DataLoader**

In [ ]:
class create_dataloader(Dataset):

    def __init__(self, csv_file, root_dir, transform):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir, self.data.fn[idx])
        try:
          image = io.imread(img_name)
        except:
          image = np.empty([64, 64, 3])
        label = map_label(self.data.label[idx])
        # image = torch.from_numpy(image)
        image = self.transform(image)
        sample = {'image': image, 'label': label}

        return sample

In [ ]:
transform = transforms.Compose(
                        [transforms.ToTensor()
                        ])

train_dataset = create_dataloader(csv_file=csv_file_path,
                                  root_dir=train_data_path,
                                  transform=transform)

training_data = DataLoader(dataset=train_dataset, batch_size=4, shuffle=True)

## **Neural Network**

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop1 = nn.Dropout(p=0.25)
        self.drop2 = nn.Dropout(p=0.50)
        self.fc1 = nn.Linear(10816, 512)
        self.fc2 = nn.Linear(512, 4)

    def forward(self, x):
      x = self.conv1(x)
      x = F.relu(x)
      x = self.conv2(x)
      x = F.relu(x)
      x = self.pool(x)
      x = self.drop1(x)
      x = self.conv3(x)
      x = F.relu(x)
      x = self.conv4(x)
      x = F.relu(x)
      x = self.pool(x)
      x = self.drop1(x)
      x = x.view(x.size(0), -1)
      x = self.fc1(x)
      x = F.relu(x)
      x = self.drop2(x)
      x = self.fc2(x)
      x = F.softmax(x)

      return x

## **Initialize Parameters**

In [ ]:
''' Network '''
net = Net().to(device)

''' Loss Function '''
criterion = nn.CrossEntropyLoss()

''' Optimizer '''
optimizer = optim.RMSprop(net.parameters(), lr=0.0001)

## **Training**

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch in enumerate(training_data):
        # get the inputs; data is a list of [inputs, labels]
        image= batch['image']
        label= batch['label']
        image = image.to(device)
        label = label.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        logits = net(image.float())
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()

        print('Epoch Number -> {} Iteration Number: {} | Loss: {}'.format(epoch, i+1, loss))

    PATH = os.path.join(save_model, 'train_net.pth')
    torch.save(net.state_dict(), PATH)

print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Streaming output truncated to the last 5000 lines.
Epoch Number -> 1 Iteration Number: 7227 | Loss: 1.3564581871032715
Epoch Number -> 1 Iteration Number: 7228 | Loss: 1.0611462593078613
Epoch Number -> 1 Iteration Number: 7229 | Loss: 0.9031636714935303
Epoch Number -> 1 Iteration Number: 7230 | Loss: 1.0597567558288574
Epoch Number -> 1 Iteration Number: 7231 | Loss: 1.0927565097808838
Epoch Number -> 1 Iteration Number: 7232 | Loss: 1.0651930570602417
Epoch Number -> 1 Iteration Number: 7233 | Loss: 0.9936486482620239
Epoch Number -> 1 Iteration Number: 7234 | Loss: 0.743739128112793
Epoch Number -> 1 Iteration Number: 7235 | Loss: 0.8903303146362305
Epoch Number -> 1 Iteration Number: 7236 | Loss: 1.0787036418914795
Epoch Number -> 1 Iteration Number: 7237 | Loss: 1.288050651550293
Epoch Number -> 1 Iteration Number: 7238 | Loss: 0.7750163674354553
Epoch Number -> 1 Iteration Number: 7239 | Loss: 1.0938811302185059
Epoch Number -> 1 Iteration Number: 7240 | Loss: 1.1626948118209839

## **Testing**

In [ ]:
PATH = os.path.join(save_model, 'train_net.pth')

net = Net().to(device)
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
transform = transforms.Compose(
                        [transforms.ToTensor()
                        ])

In [ ]:
test_image_array = list()

predicted_label = list()

for image_name in test_image_list:
  img = os.path.join(test_path, image_name)
  img = io.imread(img)
  image = transform(img)
  image = image.to(device)

  image = torch.reshape(image, (1, image.shape[0], image.shape[1], image.shape[2]))

  outputs = net(image)
  _, predicted = torch.max(outputs, 1)
  pred_label = predicted.item()
  label = return_label(pred_label)
  predicted_label.append(label)

  # rotated_right
  if pred_label==0:
    corrected_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
  # rotated_left
  if pred_label==1:
    corrected_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
  # upside_down
  if pred_label==3:
    corrected_img = cv2.rotate(img, cv2.ROTATE_180)
  # upright
  if pred_label==2:
    corrected_img = img

  cv2.imwrite(os.path.join(corrected_image_path, image_name), corrected_img)
  test_image_array.append(corrected_img)

  print(image_name)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


95-34145495_1950-08-23_2011.jpg
95-302695_1968-10-12_2013.jpg
95-31912895_1964-06-17_1996.jpg
95-32853595_1986-01-08_2015.jpg
95-30352495_1989-06-27_2013.jpg
95-3339395_1965-11-11_2001.jpg
95-32875695_1985-09-09_2009.jpg
95-3203495_1964-10-10_1983.jpg
95-32001395_1988-11-23_2011.jpg
95-33554195_1986-09-21_2012.jpg
95-30983095_1991-05-02_2013.jpg
95-33563895_1990-01-17_2012.jpg
95-34186995_1986-03-27_2011.jpg
95-335195_1964-05-13_2014.jpg
95-29736595_1986-02-17_2010.jpg
95-29409395_1992-02-26_2010.jpg
95-2995995_1966-12-19_2015.jpg
95-33734995_1990-03-24_2012.jpg
95-34802495_1980-11-06_2012.jpg
95-3483095_1986-09-08_2014.jpg
95-3493695_1976-04-28_2008.jpg
95-30122795_1989-04-16_2014.jpg
95-34127595_1973-04-30_2013.jpg
95-31329995_1940-09-14_2005.jpg
95-3475595_1984-05-23_2015.jpg
95-31396795_1987-01-09_2014.jpg
95-31152195_1987-02-11_2012.jpg
95-34428195_1994-09-22_2014.jpg
95-32535595_1937-03-02_2007.jpg
95-3190895_1926-04-04_2005.jpg
95-316095_1962-03-29_2006.jpg
95-30115695_1980-01-3

In [ ]:
# Create a dataframe from 2 lists

data_df = {'fn': image_name, 'label': predicted_label}
df = pd.DataFrame(data_df)

df.to_csv(os.path.join(path_to_save, 'test.preds.csv'), index=False)

print('File generated...')

# save numpy array of corrected test image
np.save(os.path.join(path_to_save, 'corrected_test_image_array'), test_image_array)

File generated...
